# Importing Packages and Connecting to DB

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import sys, os
import time;
import json;
from optparse import OptionParser
from cStringIO import StringIO;
from datetime import timedelta;

In [ ]:
import sys, os
import pandas as pd
import psycopg2
from datetime import timedelta
from medinfo.dataconversion import FeatureMatrixFactory as fmf;

def connectToDB():
    '''
    Connects to Database, requires password
    '''

    dsn_database = "medinfo"
    dsn_hostname = "healthrex-db.cwyfvxgvic6c.us-east-1.rds.amazonaws.com"
    dsn_port = "5432"
    dsn_uid = "jonc101"
    dsn_pwd = raw_input("Enter the database password: ")


    try:
        conn_string = "host='"+dsn_hostname+"' port="+dsn_port+" dbname='"+dsn_database+"' user='"+dsn_uid+"' password='"+dsn_pwd+"'"
        print "Connecting to database\n  ->%s" % (conn_string)
        conn=psycopg2.connect(conn_string)
        print "Connected!\n"

    except:
        print "Unable to connect to the database."

    cursor = conn.cursor()
    return cursor, conn

cursor, conn = connectToDB()

### Human Authored Order Set Validation Set
Input:

1) matrix of responses/what actually happens 24hrs after for all patient_items

2) patient_items we want to compare to from the test set of our algorithm

3) columns of responses that are comparable to order sets, where default_recommend + analysis status = 1

Output:
f1, precision, accuracy, recall

Objectives.
1) Obtain all patient items that are part of order sets

2) figure out what order sets they were a part of

3) for each order set, create a vector of responses that represent what the order set says to order

4) create a matrix of responses from this order set

5) calculate f1 and f2 score to real response variables (with filtered out clinical items as appropriate) for baseline



## Obtaining all of the patient_item_ids that are from order sets

In [ ]:
loadingdata=True

In [ ]:
#load in data
data_dir = '/home/ec2-user/cs230/scripts/HumanBaseline/Actual_Final_Files/XY_data/'
fname = data_dir+'y_dev.txt'
y_dev = pd.read_csv(fname, sep = '\t')
fname = data_dir + 'y_test.txt'
y_test = pd.read_csv(fname, sep = '\t')
response_matrix = pd.concat([y_dev, y_test])

In [ ]:
fname = data_dir+'x_dev.txt'
x_dev = pd.read_csv(fname, usecols=[0], sep = '\t')
fname = data_dir + 'x_test.txt'
x_test = pd.read_csv(fname, usecols=[0], sep = '\t')
p_id_column = pd.concat([x_dev, x_test])

In [ ]:
response_matrix["patient_item_id"] = patientitemids_from_data

In [ ]:
response_matrix.shape

In [ ]:
len(y_dev) + len(y_test)

In [ ]:
patientitemids_from_data = response_matrix["patient_item_id"]

need to change cell below based on new real columns

In [ ]:
#create response columns to use for comparison later NEED TO CHANGE THIS
response_vars = []
f = open('/home/ec2-user/cs230/scripts/matrix/response_vars_FINAL_0307.txt', 'r')
for line in f:
    response_vars.append(line.split('\n')[0])
f.close()
len(response_vars)
response_var_set = set(response_vars)
# extracts columns from all_data that we want as response variables
response_cols = [col for col in data.columns if col in response_var_set]
response_cols.insert(0, "patient_item_id")

In [ ]:
#read response_cols from csv
if loadingdata == True:
    response_cols = pd.read_csv("/home/ec2-user/cs230/scripts/HumanBaseline/response_cols.txt", sep = "\t")
    response_cols = response_cols.name.tolist()
    response_cols = [s + ".post.1d" for s in response_cols]
    response_cols.insert(0, "patient_item_id")

In [ ]:
response_matrix = response_matrix[response_cols]

In [ ]:
#load data
os.chdir('/home/ec2-user/cs230/scripts/HumanBaseline')

In [ ]:
if loadingdata == True:
    patientitemlist_pre = patients_from_ordersets_subset.patient_item_id.tolist()

In [ ]:
response_matrix.drop_duplicates(keep = "last", inplace=True)

In [ ]:
response_matrix.shape

In [ ]:
patientitemlist = ",".join(map(str, patientitemids_from_data.tolist()))
patientitemlist = "(" + patientitemlist + ")"

In [ ]:
patientItemsfromOrderSet = """
select 
		pi.patient_item_id,
        ic.external_id,
		ci.name
	from 
		item_collection as ic,
		item_collection_item as ici, 
		patient_item_collection_link as picl, 
		patient_item as pi,
		clinical_item as ci,
		clinical_item_category as cic
	where 
		pi.patient_item_id IN %s and
        ic.item_collection_id = ici.item_collection_id and
		ici.item_collection_item_id = picl.item_collection_item_id and 
		picl.patient_item_id = pi.patient_item_id and  
		pi.clinical_item_id = ci.clinical_item_id and
		ci.clinical_item_category_id = cic.clinical_item_category_id and
		ci.default_recommend = 1 and
		cic.default_recommend = 1 and
		ic.section <> 'Ad-hoc Orders' -- Existing option to include orders on the fly, even though they weren't really part of the original order set design / intent
		-- ic.external_id = 692 -- Example for Med Sepsis
    Order by pi.patient_item_id
"""%(patientitemlist) 

In [ ]:
df_patientitems = pd.read_sql(patientItemsfromOrderSet, conn)
df_patientitems.to_csv('ordersets_from_patientitems.txt', sep = '\t')
df_patientitems.head()

In [ ]:
print(df_patientitems.shape) #for 
print(len(patientitemids_from_data))
print(float(len(df_patientitems))/len(patientitemids_from_data))
#Above cell shows the percentage of patientitems that are actually order sets in general 

In [ ]:
#check to make sure patientitemids are actually in the original set
assert set(df_patientitems.patient_item_id).issubset(set(patientitemids_from_data))

In [ ]:
#if you're loading data
if loading = True:
    yhat = pd.read_table('human_authored.txt')
    y = pd.read_table('orderset_response.txt')
    assert len(yhat)==len(y)

## Collecting clinical items per external_id into a dictionary

In [ ]:
external_list = df_patientitems.external_id.unique()

In [ ]:
external_list = ",".join(map(str, external_list))
external_list = "(" + external_list + ")"

In [ ]:
externalItems = """
select 
	ic.external_id,
	ci.name
from 
	item_collection as ic,
	item_collection_item as ici, 
	clinical_item as ci,
	clinical_item_category as cic
where 
	ic.item_collection_id = ici.item_collection_id and
	ici.clinical_item_id = ci.clinical_item_id and
	ci.clinical_item_category_id = cic.clinical_item_category_id and
    ic.external_id in %s and
	ci.default_recommend = 1 and
	cic.default_recommend = 1 and
    ci.analysis_status =1 and
	ic.section <> 'Ad-hoc Orders' -- Existing option to include orders on the fly, even though they weren't really part of the original order set design / intent
group by 
	ic.external_id,
	ci.name
order by name
"""%(external_list)

In [ ]:
df_externalitems = pd.read_sql(externalItems, conn)

In [ ]:
external_id_to_items = {k: list(v) for k,v in df_externalitems.groupby("external_id")["name"]}

In [ ]:
#CHANGE THIS
#with open("ResponseCols.txt") as f:
#    responsecols = f.readlines()
#responsecols = [x.strip() for x in responsecols] 
responsecols = pd.read_csv("/home/ec2-user/cs230/scripts/HumanBaseline/response_cols.txt", sep = "\t")


In [ ]:
responsecols = responsecols.name.tolist()

In [ ]:
import numpy as np
external_id_to_vec = {}
for external_id, items in external_id_to_items.iteritems():
    z = np.zeros(len(responsecols))
    for item in items:
        if (item not in responsecols):
            continue
        z[responsecols.index(item)] = 1
    external_id_to_vec[external_id] = z
print(len(external_id_to_vec[625]))
print(sum(external_id_to_vec[625]))
# i need to create a dictionary of external ids and response vectors with response cols as the index of them

## Create a matrix with all the patient_item_ids and their response vectors

In [ ]:
df_patientitems = df_patientitems.drop_duplicates(subset = "patient_item_id", keep = "last")

In [ ]:
df_humanmatrix = df_patientitems["external_id"].map(external_id_to_vec)

In [ ]:
df_humanmatrix = pd.DataFrame(df_humanmatrix)

In [ ]:
df_humanmatrix["patient_item_id"] = df_patientitems.patient_item_id

In [ ]:
comparable_response = response_matrix[response_matrix["patient_item_id"].isin(df_humanmatrix["patient_item_id"])]

In [ ]:
comparable_response.drop_duplicates(inplace = True, keep = "last", subset = "patient_item_id")

In [ ]:
print(comparable_response.shape)
print(comparable_response.shape)
print(df_humanmatrix.size)
print(df_humanmatrix.shape)

In [ ]:
df_humanmatrix

In [ ]:
df_humanmatrix.to_pickle("humanmatrix_14_trim.p")

In [ ]:
response_cols

In [ ]:
df_humanmatrix2 = pd.DataFrame(columns = response_cols)


In [ ]:
df_humanmatrix2["patient_item_id"] = df_humanmatrix["patient_item_id"]

In [ ]:
for row in df_humanmatrix

In [ ]:
comparable_response = response_matrix[response_matrix["patient_item_id"].isin(x_test["patient_item_id"])]
df_humanmatrix = df_humanmatrix[df_humanmatrix["patient_item_id"].isin(x_test["patient_item_id"])]

In [ ]:
df_humanmatrix = df_humanmatrix.sort_values(by="patient_item_id")
comparable_response = comparable_response.sort_values(by="patient_item_id")

In [ ]:
assert sum(comparable_response.patient_item_id.values == df_humanmatrix.patient_item_id.values) == len(df_humanmatrix)

In [ ]:
df_humanmatrix.to_csv('human_authored_14_nontrim_test.txt', sep='\t')
comparable_response.to_csv('orderset_response_14_nontrim_test.txt', sep = '\t')

## analyze script for the human baseline model

In [ ]:
%matplotlib inline
from sklearn.metrics import precision_recall_fscore_support as pr
from sklearn import metrics

In [ ]:
comparable_response.shape

In [ ]:
df_humanmatrix.shape

In [ ]:
os.getcwd()

In [ ]:
df_humanmatrix = pd.read_pickle("humanmatrix_14_trim.p")

In [ ]:
comparable_response = pd.read_csv("orderset_response_14_nontrim.txt",sep='\t')

In [ ]:
comparable_response = comparable_response.drop("patient_item_id", axis = 1)
df_humanmatrix = df_humanmatrix.drop('patient_item_id', axis = 1)

In [ ]:
comparable_response.drop("Unnamed: 0", axis=1, inplace=True)

In [ ]:
y = []
y = np.asarray(comparable_response).reshape((1452*len(comparable_response),1))

In [ ]:
flattened = [val for sublist in df_humanmatrix.external_id for val in sublist]
yhat = flattened

In [ ]:
y = [1 if x > 0 else 0 for x in y]

In [ ]:
assert len(y) == len(yhat)

In [ ]:
bPrecis, bRecall, bFscore, bSupport = pr(y, yhat, pos_label =1, average='binary')
print(bPrecis)
print(bRecall)
print(bFscore)
print(bSupport)

In [ ]:
def plot_roc_curve(y,yhat,title,text,y_probs=[]):
    '''
    Plots a ROC curve for given set of predictions
    
    parameters:
        @predict_results: predict generator object
    return:
        None
    '''
        
    fpr, tpr, _ = metrics.roc_curve(y,yhat)
    print(fpr)
    print(tpr)
    AUROC = metrics.auc(fpr,tpr)
    plt.plot(fpr,tpr,label=text + ' (AUC = %.3f)' % AUROC)
    plt.legend(loc='lower right')
    plt.title(title)
    plt.xlabel('1 - Specificity')
    plt.ylabel('Sensitivity')
    plt.show()
    return

In [ ]:
plot_roc_curve(y, yhat,"Human Authored ROC curve", "swag")

In [ ]:
metrics.accuracy_score(y,yhat)

In [ ]:
len(y)

In [ ]:
thefile = open('human_response.txt', 'w')
for item in y:
    thefile.write("%s\n" % item)

In [ ]:
thefile.close()

In [ ]:
thefile2 = open('human_prediction.txt', 'w')
for item in yhat:
    thefile2.write("%s\n" % item)
thefile2.close()

In [ ]:
len(yhat)

In [ ]:
os.getcwd()

In [ ]:
df_humanmatrix_real = pd.DataFrame(index = range(60782), columns = comparable_response.columns)

In [ ]:
df_humanmatrix_real.shape

In [ ]:
list_swag = df_humanmatrix.external_id.tolist()

In [ ]:
list_swag

In [ ]:
df_humanmatrix_real = pd.DataFrame(list_swag, columns = comparable_response.columns)

In [ ]:
df_humanmatrix_real.shape

In [ ]:
sum(pd.Series(comparable_response.values.ravel()).isnull())

In [ ]:
df_humanmatrix_real.to_csv("humanpredict_matrix_real.txt", sep = '\t')

In [ ]:
comparable_response.to_csv('comparable_response.txt', sep='\t')

Individual clinical item statistics, finding stats for every column individually

In [ ]:
response_names = comparable_response.columns
results = pd.DataFrame(columns = ['Clinical_Item', 'F1_Score', "precision", "recall", "AUROC", "accuracy"])

for response_num in range(1452):
    y_ = comparable_response.iloc[:,response_num]
    yhat = df_humanmatrix_real.iloc[:,response_num]
    bPrecis_, bRecall_, bFscore_, _ = pr(y_, yhat, average='binary')
    results.loc[response_num, 'Clinical_Item'] = response_names[response_num]
    results.loc[response_num, 'F1_Score'] = bFscore_
    results.loc[response_num, 'precision']=bPrecis_
    results.loc[response_num, 'recall'] = bRecall_
    fpr, tpr, _ = metrics.roc_curve(y_,yhat)
    AUROC = metrics.auc(fpr,tpr)
    results.loc[response_num, 'AUROC'] = AUROC
    results.loc[response_num, 'accuracy'] = metrics.accuracy_score(y_,yhat)
    if response_num%100 ==0:
        print "Clinical Item: %s \t results:" % (response_names[response_num])+ str(results.loc[response_num, :])

In [ ]:
results.sort_values("F1_Score", ascending = False)

In [ ]:
results.to_csv("results_for_humanauthored.txt", sep = '\t')

### list of patient_items from order_sets, and the associated number of items predicted in that order set

In [ ]:
response_cols = """
	select 
		distinct ci.name
	from 
		clinical_item as ci,
		clinical_item_category as cic
	where 
		ci.clinical_item_category_id = cic.clinical_item_category_id and
		ci.default_recommend = 1 and
		cic.default_recommend = 1 and
        ci.analysis_status = 1 
"""

In [ ]:
df_response_cols = pd.read_sql(response_cols, conn)

In [ ]:
df_response_cols.to_csv("response_cols.txt", sep = '\t')

In [ ]:
len(df_response_cols)

In [ ]:
response_list = ",".join(map(str, df_response_cols.name))
response_list = "(" + response_list + ")"

In [ ]:
#select patient_item list with external_id
#then select external_id from these patient item lists and include the tables for ci.name and distinct count ci.name for each external_id from this list

patient_items_num_in_orderset = """
	select 
		ic.external_id,
		pi.patient_item_id
	from 
		item_collection as ic,
		item_collection_item as ici, 
		patient_item_collection_link as picl, 
		patient_item as pi,
		clinical_item as ci,
		clinical_item_category as cic
	where 
		ic.item_collection_id = ici.item_collection_id and
		ici.item_collection_item_id = picl.item_collection_item_id and 
		picl.patient_item_id = pi.patient_item_id and  
		pi.clinical_item_id = ci.clinical_item_id and
		ci.clinical_item_category_id = cic.clinical_item_category_id and
		ci.default_recommend = 1 and
		cic.default_recommend = 1 and
        ci.analysis_status = 1 and
		ic.section <> 'Ad-hoc Orders' -- Existing option to include orders on the fly, even though they weren't really part of the original order set design / intent
"""

In [ ]:
df_patientitems_orderset_num = pd.read_sql(patient_items_num_in_orderset, conn)

In [ ]:
len(df_patientitems_orderset_num)

In [ ]:
all_externalitems_num = """
select 
	ic.external_id,
	count(distinct ci.name)
from 
	item_collection as ic,
	item_collection_item as ici, 
	clinical_item as ci,
	clinical_item_category as cic
where 
	ic.item_collection_id = ici.item_collection_id and
	ici.clinical_item_id = ci.clinical_item_id and
	ci.clinical_item_category_id = cic.clinical_item_category_id and
	ci.default_recommend = 1 and
	cic.default_recommend = 1 and
	ic.section <> 'Ad-hoc Orders' -- Existing option to include orders on the fly, even though they weren't really part of the original order set design / intent
group by 
	ic.external_id
"""

In [ ]:
df_externalitems_num = pd.read_sql(all_externalitems_num, conn)

In [ ]:
df_patientitems_orderset_num["ordersetsize"] = ""

In [ ]:
df_externalitems_num["external_id"][0]

In [ ]:
df_patientitems_orderset_num.drop("ordersetsize", axis = 1, inplace=True)

In [ ]:
df_patientitems_final = pd.merge(df_patientitems_orderset_num, df_externalitems_num, on = "external_id")

In [ ]:
df_patientitems_final.to_csv("patientitems_num_orderset.txt", sep = '\t')

### get list of patient_item_ids from order sets based on the file delaney sent

In [ ]:
data_dir = '/home/ec2-user/cs230/scripts/matrix/final_data_031618_v2/'

In [ ]:
fname = data_dir + "final_data_dropcols_"+ str(0) + ".txt"
data = pd.read_table(fname, usecols=[0], sep = '\t', header = 2)

In [ ]:
for i in range(1, 15):
    fname = data_dir + "final_data_dropcols_"+ str(i) + ".txt"
    temp = pd.read_table(fname, usecols=[0], sep = '\t', header = 2)
    data = pd.concat([temp, data])

In [ ]:
len(data.patient_item_id.unique())

In [ ]:
full_list_patients = pd.read_csv("patientitems_num_orderset.txt", sep = '\t')

In [ ]:
thelist = data.patient_item_id.tolist()
thefile = open('patient_item_id_list_full_notrim_14.txt', 'w')
for item in thelist:
  thefile.write("%s\n" % item)

In [ ]:
patients_from_ordersets_subset = full_list_patients.merge(data, on = "patient_item_id", how = "inner")

In [ ]:
len(patients_from_ordersets_subset)

In [ ]:
patients_from_ordersets_subset.to_csv("patients_from_ordersets_subset_notrim_14.txt", sep='\t')

## Below are some example Queries
Some interesting facts about the data we have:
There are 2476261 total order_set usages from patient_items out of 55470744 patient_items (4 percent)

In [ ]:
 orderSet_overall_usage = """select 
		count(distinct pi.patient_item_id)
	from 
		item_collection as ic,
		item_collection_item as ici, 
		patient_item_collection_link as picl, 
		patient_item as pi,
		clinical_item as ci,
		clinical_item_category as cic
	where 
		ic.item_collection_id = ici.item_collection_id and
		ici.item_collection_item_id = picl.item_collection_item_id and 
		picl.patient_item_id = pi.patient_item_id and  
		pi.clinical_item_id = ci.clinical_item_id and
		ci.clinical_item_category_id = cic.clinical_item_category_id and
		ci.default_recommend = 1 and
		cic.default_recommend = 1 and
		ic.section <> 'Ad-hoc Orders' -- Existing option to include orders on the fly, even though they weren't really part of the original order set design / intent
		-- ic.external_id = 692 -- Example for Med Sepsis
        """

In [ ]:
total_patient_items = """select count(distinct patient_item_id) from patient_item"""

In [ ]:
cursor.execute(total_patient_items)

In [ ]:
cursor.fetchall()

In [ ]:
recommend_clinical = """
select 
	count(distinct ci.name)
from 
	clinical_item as ci,
	clinical_item_category as cic
where 
	ci.clinical_item_category_id = cic.clinical_item_category_id and
	ci.default_recommend = 1 and
	cic.default_recommend = 1 and
    ci.analysis_status=1 
"""
cursor.execute(recommend_clinical)
cursor.fetchall()

In [ ]:
 orderSetComponentUsage_default = """select 
		ic.external_id,
		ic.name,
		ci.clinical_item_id,
		ci.name,
		ci.description,
		count(distinct pi.patient_item_id) as itemCount
	from 
		item_collection as ic,
		item_collection_item as ici, 
		patient_item_collection_link as picl, 
		patient_item as pi,
		clinical_item as ci,
		clinical_item_category as cic
	where 
		ic.item_collection_id = ici.item_collection_id and
		ici.item_collection_item_id = picl.item_collection_item_id and 
		picl.patient_item_id = pi.patient_item_id and  
		pi.clinical_item_id = ci.clinical_item_id and
		ci.clinical_item_category_id = cic.clinical_item_category_id and
		ci.default_recommend = 1 and
		cic.default_recommend = 1 and
		ic.section <> 'Ad-hoc Orders' -- Existing option to include orders on the fly, even though they weren't really part of the original order set design / intent
		-- ic.external_id = 692 -- Example for Med Sepsis
	group by 
		ic.external_id,
		ic.name,
		ci.clinical_item_id,
		ci.name,
		ci.description
        """

In [ ]:
cursor.execute(orderSetComponentUsage_default)

In [ ]:
cursor.fetchall()

In [ ]:
orderSetUsage_Default = """select external_id, name, count(patient_id) as patientCount, 
   avg(itemCount) as avgItemCount, min(itemCount) as minItemCount, max(itemCount) as maxItemCount,
   sum(itemCount) as sumItemCount, sum(itemCount*itemCount) as sumSquaresItemCount
from
(
	select 
		ic.external_id,
		ic.name,
		pi.patient_id,
		count(distinct pi.clinical_item_id) as itemCount
	from 
		item_collection as ic,
		item_collection_item as ici, 
		patient_item_collection_link as picl, 
		patient_item as pi,
		clinical_item as ci,
		clinical_item_category as cic
	where 
		ic.item_collection_id = ici.item_collection_id and
		ici.item_collection_item_id = picl.item_collection_item_id and 
		picl.patient_item_id = pi.patient_item_id and  
		pi.clinical_item_id = ci.clinical_item_id and
		ci.clinical_item_category_id = cic.clinical_item_category_id and
		ci.default_recommend = 1 and
		cic.default_recommend = 1 and
		ic.section <> 'Ad-hoc Orders' -- Existing option to include orders on the fly, even though they weren't really part of the original order set design / intent
	group by 
		ic.external_id,
		ic.name,
		pi.patient_id
) as orderSetPatient,
clinical_ite
group by 
	external_id,
	name
"""

In [ ]:
cursor.execute(orderSetUsage_Default)

In [ ]:
cursor.fetchall()

In [ ]:
OrderSetSize_default = """
select 
	ic.external_id,
	ic.name,
	count(distinct ici.clinical_item_id) as itemCount
from 
	item_collection as ic,
	item_collection_item as ici, 
	clinical_item as ci,
	clinical_item_category as cic
where 
	ic.item_collection_id = ici.item_collection_id and
	ici.clinical_item_id = ci.clinical_item_id and
	ci.clinical_item_category_id = cic.clinical_item_category_id and
	ci.default_recommend = 1 and
	cic.default_recommend = 1 and
	ic.section <> 'Ad-hoc Orders' -- Existing option to include orders on the fly, even though they weren't really part of the original order set design / intent
group by 
	ic.external_id,
	ic.name
"""

In [ ]:
cursor.execute(OrderSetSize_default)

In [ ]:
cursor.fetchall()

In [ ]:
patients_from_ordersets_subset